In [1]:
%cd ../../..


/run/media/nazif/2F946E411BA61D49/mirscribe


In [2]:
import pandas as pd
import concurrent.futures
from scripts.utils import *
from scripts.ensembl import *

g37 = import_pyensembl(37)

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


In [3]:
def get_transcript_id(coord):
    if transcript_id := g37.transcript_ids_at_locus(*coord):
        return tuple(coord), transcript_id
    else:
        return tuple(coord), "not_found"

In [5]:
relevant_cols = ["id", "chr", "start_coordinate", "end_coordinate", "mirna_accession", "is_mutated", "prediction", "binary_prediction", "pred_difference", "pred_difference_binary"]
filename = "results/from_truba/vcf1_results_0_2000_with_prediction_only_meaningful_results"

df = pd.read_csv(f"{filename}.csv", low_memory=False, usecols=relevant_cols)

2.0

In [11]:
import pandas as pd

df['id'] = df['id'].str.replace('PD3851a', 'PD3851a_', regex=False)
df['chr'] = df['chr'].str.replace('PD3851a', '', regex=False)



In [12]:
df.head()

,mirna_accession,is_mutated,id,chr,start_coordinate,end_coordinate,prediction,binary_prediction,pred_difference,pred_difference_binary,ENST
0,MIMAT0000069,0,PD3851a__10_103447203_C_A_MIMAT0000069,10,103447188,103447197,0.312751,0,0.328268,1,not_found
1,MIMAT0000069,1,PD3851a__10_103447203_C_A_MIMAT0000069,10,103447188,103447197,0.641019,1,0.328268,1,not_found
2,MIMAT0000241,0,PD3851a__10_103447203_C_A_MIMAT0000241,10,103447202,103447215,0.793109,1,-0.602264,-1,not_found
3,MIMAT0000241,1,PD3851a__10_103447203_C_A_MIMAT0000241,10,103447173,103447193,0.190845,0,-0.602264,-1,not_found
4,MIMAT0000257,0,PD3851a__10_103447203_C_A_MIMAT0000257,10,103447210,103447222,0.434621,0,0.232544,1,not_found


In [13]:
# adding ENST IDs

coords = df[['chr', 'start_coordinate']].values.tolist()

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(get_transcript_id, coords)
    
ensts = dict(results)
df["ENST"] = [ensts.get((row["chr"], row["start_coordinate"]), "") for _, row in df.iterrows()]

# report
print(f"{len(df[df.ENST == 'not_found']) / len(df):.3%} of the coordinates have no transcripts")


51.562% of the coordinates have no transcripts


In [14]:
cols = ["id", "mirna_accession", "ENST", "is_mutated", "pred_difference_binary"]


gain_df = df[cols][(df[cols].is_mutated == 0) & (df[cols].pred_difference_binary == 1)] 
loss_df = df[cols][(df[cols].is_mutated == 0) & (df[cols].pred_difference_binary == -1)] 

gain_df.drop(columns=["is_mutated", "pred_difference_binary"], inplace=True)
loss_df.drop(columns=["is_mutated", "pred_difference_binary"], inplace=True)

# remove the last 13 characters (MIMAT0000070 like identifiers)
gain_df['id'] = gain_df['id'].str[:-13]
loss_df['id'] = loss_df['id'].str[:-13]

In [15]:
gain_df["gain"] = 1
gain_df["loss"] = 0
loss_df["gain"] = 0
loss_df["loss"] = 1
final_df = pd.concat([gain_df, loss_df], ignore_index=True)

In [16]:
final_df.to_csv("results/from_truba/vcf1_results.csv", index=False)